In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wiki-data2/all_wiki_data.csv
/kaggle/input/wiki-data2/test_wiki_data.csv
/kaggle/input/wiki-data2/train_wiki_data.csv


In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
import pandas as pd

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [4]:
train_data = pd.read_csv('/kaggle/input/wiki-data2/train_wiki_data.csv',encoding='utf-8', delimiter=',')
test_data = pd.read_csv('/kaggle/input/wiki-data2/test_wiki_data.csv',encoding='utf-8', delimiter=',')

In [5]:
train_data['label'].value_counts()

label
0    1115
1     345
3     218
2     146
Name: count, dtype: int64

In [6]:
import re
import nltk
from stop_words import get_stop_words

nltk.download('stopwords')
stop_words = get_stop_words('vi')

def preprocess_text(text):
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\([^()]*\)', '', text)
    text = re.sub(r'\xa0', ' ',text)
#     words = text.split()
#     filtered_words = [word for word in words if word.lower() not in stop_words]
#     filtered_text = ' '.join(filtered_words)
    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
train_data['content'] = train_data['content'].apply(preprocess_text)
test_data['content'] = test_data['content'].apply(preprocess_text)

In [8]:
num_samples_to_remove = 600
indices_to_remove = train_data[train_data['label'] == 0].sample(n=num_samples_to_remove, random_state=42).index
train_data = train_data.drop(indices_to_remove)

In [9]:
train_data['label'].value_counts()

label
0    515
1    345
3    218
2    146
Name: count, dtype: int64

In [10]:
train_data['label'].isnull().value_counts()

label
False    1224
Name: count, dtype: int64

In [11]:
train_data = train_data.reset_index(drop=True)

In [12]:
train_data

,title,content,label
0,Paleauletobius silenus,Paleauletobius silenus là một loài bọ cánh cứn...,0
1,Dundahera,Dundahera là một thị trấn thống kê của quận G...,1
2,Nạn khan hiếm nhu yếu phẩm tại Venezuela,Nạn khan hiếm nhu yếu phẩm tại Venezuela được...,1
3,Archidendron royenii,Archidendron royenii là một loài thực vật có h...,0
4,"Laurens, Hérault",Laurens là một xã thuộc tỉnh Hérault trong vùn...,1
...,...,...,...
1219,Thể thao cá nhân,Thuật ngữ nhóm các môn thể thao cá nhân dùng đ...,3
1220,Ba môn phối hợp,"Ba môn phối hợp bao gồm chạy bộ , bơi và đua ...",3
1221,Trò chơi bài,Một trò chơi bài là bất kỳ trò chơi nào sử dụn...,3
1222,Thể thao trí tuệ,Thể thao trí tuệ là loại thể thao tập trung và...,3


In [13]:
test_data

,title,content,label
0,Pholcus mengla,Pholcus mengla là một loài nhện trong họ Pholc...,0
1,Sideridis pseudoyu,Sideridis pseudoyu là một loài bướm đêm trong ...,0
2,Quasimus geminus,Quasimus geminus là một loài bọ cánh cứng tron...,0
3,Lepanthes saltatrix,Lepanthes saltatrix là một loài thực vật có ho...,0
4,Glenea elongatipennis,Glenea elongatipennis là một loài bọ cánh cứng...,0
...,...,...,...
395,Victor Hugo,Victor-Marie Hugo ( tiếng Pháp: [viktɔʁ maʁi y...,3
396,YouTuber,Một YouTuber là một nhân vật trực tuyến và/hoặ...,3
397,Danh sách đĩa đơn bán chạy nhất thế giới,Danh sách các đĩa đơn bán chạy nhất liệt kê cá...,3
398,Tabloid Junkie,""" Tabloid Junkie "" là ca khúc nhạc pop do nghệ...",3


In [14]:
test_data['content']

0      Pholcus mengla là một loài nhện trong họ Pholc...
1      Sideridis pseudoyu là một loài bướm đêm trong ...
2      Quasimus geminus là một loài bọ cánh cứng tron...
3      Lepanthes saltatrix là một loài thực vật có ho...
4      Glenea elongatipennis là một loài bọ cánh cứng...
                             ...                        
395    Victor-Marie Hugo ( tiếng Pháp: [viktɔʁ maʁi y...
396    Một YouTuber là một nhân vật trực tuyến và/hoặ...
397    Danh sách các đĩa đơn bán chạy nhất liệt kê cá...
398    " Tabloid Junkie " là ca khúc nhạc pop do nghệ...
399    Morenatsu-漏れなつ。 là 1 Adult game của Nhật Bản ....
Name: content, Length: 400, dtype: object

In [15]:
print(len(train_data["content"]))
print(len(train_data["label"]))

1224
1224


In [16]:
print(train_data["content"].isnull().sum())
print(train_data["label"].isnull().sum())

0
0


In [17]:
# Define training parameters
MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001

In [18]:
# Define dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        content = str(self.data.loc[index, 'content'])
        label = self.data.loc[index, 'label']
        
        encoding = self.tokenizer.encode_plus(
            content,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'content': content,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [19]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModel.from_pretrained("vinai/phobert-base")
# Move model to GPU
model.to(device)

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [20]:
# Define additional layers for classification
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [21]:
# Prepare DataLoader
train_dataset = CustomDataset(train_data, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = CustomDataset(test_data, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [22]:
# Initialize classifier
num_labels = 4
classifier = Classifier(input_dim=model.config.hidden_size, hidden_dim=64, output_dim=num_labels)

# Move classifier to GPU
classifier.to(device)

Classifier(
  (fc1): Linear(in_features=768, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=4, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [23]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

In [24]:
import torch
from sklearn.metrics import classification_report, accuracy_score

# Training loop
for epoch in range(EPOCHS):
    # Train
    classifier.train()
    train_loss = 0.0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            last_hidden_states = outputs.last_hidden_state[:, 0, :]  # CLS token

        logits = classifier(last_hidden_states)
        loss = loss_fn(logits, labels)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * input_ids.size(0)

    # Evaluate
    classifier.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            last_hidden_states = outputs.last_hidden_state[:, 0, :]
            
            logits = classifier(last_hidden_states)
            loss = loss_fn(logits, labels)
            val_loss += loss.item() * input_ids.size(0)
            
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    train_loss /= len(train_data)
    val_loss /= len(test_data)
    report = classification_report(all_labels, all_preds)

    # Calculate percentage of wrong predictions for each label
    wrong_percentage = {}
    for label in range(num_labels):  # Assuming num_classes is defined somewhere
        # Count the number of samples misclassified for this label
        wrong_count = sum(1 for true_label, pred_label in zip(all_labels, all_preds) if true_label == label and pred_label != label)
        # Calculate the percentage
        wrong_percentage[label] = (wrong_count / all_labels.count(label)) * 100 if all_labels.count(label) > 0 else 0

    print(f'Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    print("Classification Report:")
    print(report)
    print("Percentage of wrong predictions for each label:")
    print(wrong_percentage)

Epoch 1/10, Train Loss: 0.8522, Val Loss: 0.6108
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       120
           1       0.93      0.75      0.83       120
           2       0.86      0.45      0.59        80
           3       0.54      0.95      0.69        80

    accuracy                           0.79       400
   macro avg       0.82      0.77      0.76       400
weighted avg       0.84      0.79      0.79       400

Percentage of wrong predictions for each label:
{0: 5.0, 1: 25.0, 2: 55.00000000000001, 3: 5.0}
Epoch 2/10, Train Loss: 0.4669, Val Loss: 0.4849
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       120
           1       0.91      0.77      0.83       120
           2       0.83      0.66      0.74        80
           3       0.60      0.86      0.71        80

    accuracy                           0.82       400